### Recursive functions
A recursive call is said to be in tail position if the caller only returns the value of the recursive call but does not do anything else with it. As an example, consider a simple function computing factorial.

In [1]:
def factorial(n: Int): Int = {
    n match {
        case 0 => 1
        case _ => n*factorial(n - 1) // This call is not in tail position
    }
}

val testValue = 5
print(s"Factorial of ${testValue} is: ${factorial(testValue)}")

Factorial of 5 is: 120

defined function factorial
testValue: Int = 5

A better implementation is one where the recursive call is in tail position:

In [2]:
def factorial2(n: Int): Int = {
    // `@annotation.tailrec` forces Scala to compile code that eliminates the current function call from stack 
    // when the recursive call is made (tail call optimization).
    // Optimization is made automatically, but annotation ensures that compiler throws error for 
    // a head recursive function.
    @annotation.tailrec 
    def go(n: Int, acc: Int): Int =
        if (n<=0) acc
        else go(n-1, n*acc) // This is in tail position!
    go(n, 1)
}

print(s"Factorial of ${testValue} is: ${factorial2(testValue)}")

Factorial of 5 is: 120

defined function factorial2

### Exercise 2.1
Write tail recursive implementation to compute `n`th Fibonacci number.

In [3]:
// First write a naive implementation
def fib(n: Int): Int = {
    n match {
        case 0 => 0
        case 1 => 1
        case _ => fib(n-1) + fib(n-2) // Not in tail position!
    }
}

val firstTen = (0 until 10).map(fib)

print(s"First ten Fibonacci numbers are: ${firstTen mkString ", "}")


First ten Fibonacci numbers are: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34

defined function fib
firstTen: collection.immutable.IndexedSeq[Int] = Vector(0, 1, 1, 2, 3, 5, 8, 13, 21, 34)

In [4]:
// Tail-recursive implementation that traverses the Fibonacci sequence starting from beginning
def fib2(n: Int): Int = {
    @annotation.tailrec
    def proceedInFib(iter: Int, previousValue: Int, currentValue: Int): Int = {
        if (iter <= 0) currentValue
        else proceedInFib(iter-1, previousValue = currentValue, currentValue = previousValue + currentValue)
    }
    proceedInFib(iter = n, previousValue = 0, currentValue = 1)
}

print(s"First ten Fibonacci numbers are: ${firstTen mkString ", "}")


First ten Fibonacci numbers are: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34

defined function fib2

### Higher-order functions
Example of a function that accepts a function as argument. The function is polymorphic in type as it is not restricted to a given type.

In [5]:
def findFirst[A](as: Array[A], p: A => Boolean): Int = {
    @annotation.tailrec
    def loop(n: Int): Int = {
        if (n >= as.length) -1
        else if (p(as(n))) n 
        else loop(n+1)
    }
    loop(0)
}
findFirst(Array(2, 3, 3, 1, 1), (x: Int) => x == 1)

defined function findFirst
res4_1: Int = 3

### Exercise 2.2
Implement `isSorted` checking if an array is sorted according to given comparison function.

In [6]:
def isSorted[A](as: Array[A], o: (A, A) => Boolean): Boolean = {
    @annotation.tailrec
    def loop(n: Int): Boolean = {
        if (n >= as.length - 1) true // Reached last value, all ok
        else o(as(n), as(n+1)) && loop(n + 1)
    }
    loop(0)
}

val as = Array(2, 3, 4, 8, 0)
val o = (x: Int, y: Int) => x < y
val as2 = as.slice(0, 4)
println(s"Array ${as mkString ","} is ${if (isSorted(as, o)) "sorted" else "not sorted"}")
println(s"Array ${as2 mkString ","} is ${if (isSorted(as2, o)) "sorted" else "not sorted"}")


Array 2,3,4,8,0 is not sorted
Array 2,3,4,8 is sorted


defined function isSorted
as: Array[Int] = Array(2, 3, 4, 8, 0)
o: (Int, Int) => Boolean = <function2>
as2: Array[Int] = Array(2, 3, 4, 8)

### Exercise 2.3 
Implement `currying` function with signature
```scala
def curry[A,B,C](f: (A, B) => C): A => (B => C)
```

In [7]:
def curry[A,B,C](f: (A, B) => C): A => (B => C) = {
    (a: A) => (b: B) => f(a, b)
}

def introduce(name: String, age: Int): String = s"My name is ${name} and age ${age}"
val curried = curry(introduce)
val introduceMarkAtAge = curried("Mark")

println(s"Now Mark introduces as: ${introduceMarkAtAge(15)}")
println(s"Next year Mark introduces as: ${introduceMarkAtAge(16)}")

Now Mark introduces as: My name is Mark and age 15
Next year Mark introduces as: My name is Mark and age 16


defined function curry
defined function introduce
curried: String => Int => String = <function1>
introduceMarkAtAge: Int => String = <function1>

### Exercise 2.4
Implement `uncurry` with signature
```scala
    def uncurry[A,B,C](f: A => B => C): (A, B) => C
```

In [8]:
def uncurry[A,B,C](f: A => B => C): (A, B) => C = {
    (a: A, b: B) => f(a)(b)
}

def uncurried = uncurry(curried) // `curried` defined above
println(s"Mark introduces as: ${uncurried("Mark", 15)}")

Mark introduces as: My name is Mark and age 15


defined function uncurry
defined function uncurried

### Exercise 2.5
Implement `compose`:
```scala
   def compose[A,B,C](f: B => C, g: A => B): A => C
```

In [9]:
def compose[A,B,C](f: B => C, g: A => B): A => C = {
    (a: A) => f(g(a))
}

defined function compose